In [3]:
%matplotlib inline

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import menpo.io as mio
from menpo.math import log_gabor
from menpo.feature import greyscale
from menpo.image import Image
from menpo.visualize import print_dynamic, progress_bar_str, visualize_images
from menpofit.visualize import visualize_fitting_results
from alaborticcv2015.alignment import (
    LKFitter, 
    FilteredSSD, FilteredFourierSSD, ECC, GradientImages, GradientCorrelation)
from alaborticcv2015.alignment.result import SerializableResult

# Experiment 2: YaleB Database, PCANet

In [4]:
db = mio.import_pickle('/Users/joan/Desktop/alaborticcv2015/data/yaleb.pkl.gz')

In [28]:
folder_path = '/Users/joan/Desktop/alaborticcv2015/results/yaleb/'

diagonal = 100
scales = (1,)
max_iters = 100

n_rep = 5
noise_std = [16]

## PCA Net

In [49]:
features = greyscale
landmarks = True
net_type = 'pca'
architecture = 3
n_layers = 1
n_filters = 16
patch_shape = 7
n_images = 1000

if landmarks:
    patch_type = 'landmarks'
else:
    patch_type = 'grid'
    
pcanet_params = (net_type
                + '_'
                + str(n_images)
                + '_'
                + features.__name__
                + '_'
                + str(architecture)
                + '_'
                + str(n_layers)
                + '_'
                + str(n_filters)
                + '_'
                + str(patch_shape)
                + '_'
               + patch_type)

file_name = ('/Users/joan/Desktop/alaborticcv2015/networks/'
             + pcanet_params
             + '.pkl.gz')

net = mio.import_pickle(file_name)

In [50]:
def kernel_func(shape):
    return net._compute_kernel(ext_shape=shape)

errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))
for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredFourierSSD,
                          kernel_func=kernel_func)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
# mio.export_pickle(errors, 
#                   folder_path + 'yaleb_pcanet_' + pcanet_params + '.pkl.gz', 
#                   overwrite=True)

- noise: 16,  sub: 0,  img: 9,  mean: 0.1248 - 0.1422  median: 0.1275 - 0.1370  std: 0.0334 - 0.0599 
- noise: 16,  sub: 1,  img: 9,  mean: 0.1248 - 0.1512  median: 0.1275 - 0.1440  std: 0.0334 - 0.0683 
- noise: 16,  sub: 2,  img: 9,  mean: 0.1248 - 0.1491  median: 0.1275 - 0.1430  std: 0.0334 - 0.0605 
- noise: 16,  sub: 3,  img: 9,  mean: 0.1248 - 0.1719  median: 0.1275 - 0.1752  std: 0.0334 - 0.0804 
- noise: 16,  sub: 4,  img: 1,  mean: 0.1100 - 0.0874  median: 0.1018 - 0.0765  std: 0.0179 - 0.0363 

KeyboardInterrupt: 

In [ ]:
visualize_fitting_results(fr)